# Prepare text data for evaluation : en, zh, fr, de, it, ru and es

We used [anki's](http://www.manythings.org/anki/) data because the data is almost parallel.  
[anki.sh] script downloads and extracts for the specified pair the corresponding parallel data.

In [1]:
%cd /home/jupyter/meta_XLM

/home/jupyter/meta_XLM


In [2]:
%env outpath=/home/jupyter/data/fairseq/evaluation/en
%env n_samples=100000
! chmod +x anki.sh
! chmod +x fairseq/samples.sh

env: outpath=/home/jupyter/data/fairseq/evaluation/en
env: n_samples=100000


In [ ]:
#### download and extract && rename and reduce the number of sample

In [3]:
! ./anki.sh en,zh cmn-eng $outpath anki.py && ./fairseq/samples.sh $outpath/en-zh.zh.txt $outpath/zh.txt $n_samples
! ./anki.sh en,fr fra-eng $outpath anki.py && ./fairseq/samples.sh $outpath/en-fr.fr.txt $outpath/fr.txt $n_samples
! ./anki.sh de,en deu-eng $outpath anki.py && ./fairseq/samples.sh $outpath/de-en.de.txt $outpath/de.txt $n_samples
! ./anki.sh en,it ita-eng $outpath anki.py && ./fairseq/samples.sh $outpath/en-it.it.txt $outpath/it.txt $n_samples
! ./anki.sh en,ru rus-eng $outpath anki.py && ./fairseq/samples.sh $outpath/en-ru.ru.txt $outpath/ru.txt $n_samples
! ./anki.sh en,es spa-eng $outpath anki.py && ./fairseq/samples.sh $outpath/en-es.es.txt $outpath/es.txt $n_samples

--2020-05-31 14:09:54--  http://www.manythings.org/anki/cmn-eng.zip
Resolving www.manythings.org (www.manythings.org)... 172.67.173.198, 104.24.108.196, 104.24.109.196, ...
Connecting to www.manythings.org (www.manythings.org)|172.67.173.198|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 963049 (940K) [application/zip]
Saving to: ‘/home/jupyter/data/fairseq/evaluation/en/cmn-eng.zip’

cmn-eng.zip         100%[===================>] 940.48K  3.38MB/s    in 0.3s    

2020-05-31 14:09:55 (3.38 MB/s) - ‘/home/jupyter/data/fairseq/evaluation/en/cmn-eng.zip’ saved [963049/963049]

Archive:  /home/jupyter/data/fairseq/evaluation/en/cmn-eng.zip
  inflating: /home/jupyter/data/fairseq/evaluation/en/cmn.txt  
  inflating: /home/jupyter/data/fairseq/evaluation/en/_about.txt  
download and extract cmn-eng data in /home/jupyter/data/fairseq/evaluation/en
select 100000 of /home/jupyter/data/fairseq/evaluation/en/en-zh.zh.txt to /home/jupyter/data/fairseq/evaluation/en/zh.txt

#### en data

In [4]:
! ./fairseq/samples.sh $outpath/de-en.en.txt $outpath/en.txt $n_samples

select 100000 of /home/jupyter/data/fairseq/evaluation/en/de-en.en.txt to /home/jupyter/data/fairseq/evaluation/en/en.txt 


# download models

If you want to train your one model, look https://github.com/pytorch/fairseq/tree/master/examples/language_model

In [6]:
%env tgt_path=/home/jupyter/fairseq_models
! chmod +x ./fairseq/download_model.sh

env: tgt_path=/home/jupyter/fairseq_models


In [7]:
! ./fairseq/download_model.sh en $tgt_path

--2020-05-31 14:11:23--  https://dl.fbaipublicfiles.com/fairseq/models/lm/wmt19.en.tar.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 2606:4700:10::6816:4a8e, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3590918000 (3.3G) [application/gzip]
Saving to: ‘/home/jupyter/fairseq_models/wmt19.en.tar.gz’

wmt19.en.tar.gz     100%[===================>]   3.34G  11.6MB/s    in 2m 29s  

2020-05-31 14:13:52 (23.0 MB/s) - ‘/home/jupyter/fairseq_models/wmt19.en.tar.gz’ saved [3590918000/3590918000]

wmt19.en/
wmt19.en/model.pt
wmt19.en/dict.txt
wmt19.en/bpecodes


# install dependencies and clone fairseq if not already done

! pip install fastBPE sacremoses  
! git clone https://github.com/pytorch/fairseq /home/jupyter/fairseq

In [8]:
%cd /home/jupyter/fairseq

/home/jupyter/fairseq


In [11]:
#%env modelpath=$tgt_path/wmnt19.en
%env modelpath=/home/jupyter/fairseq_models/wmt19.en
%env destdir=/home/jupyter/fairseq_data/en
%env src_lang=en

env: modelpath=/home/jupyter/fairseq_models/wmt19.en
env: destdir=/home/jupyter/fairseq_data/en
env: src_lang=en


#### en vs en

In [12]:
%env tgt_lang=en
# Processes data (only --testpref)
! fairseq-preprocess --only-source --testpref $outpath/$tgt_lang.txt --destdir $destdir/$src_lang-$tgt_lang --workers 20 --joined-dictionary --srcdict $modelpath/dict.txt
# run evalution
! fairseq-eval-lm $destdir/$src_lang-$tgt_lang --path $modelpath/model.pt --max-sentences 2 --tokens-per-sample 512 --context-window 400

env: tgt_lang=en
2020-05-31 14:20:18 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, bf16=False, bpe=None, checkpoint_suffix='', cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='/home/jupyter/fairseq_data/en/en-en', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=True, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=True, optimizer='nag', padding_factor=8, quantization_config_path=None, seed=1, source_lang=None, srcdict='/home/jupyter/fairseq_models/wmt19.en/dict.txt', target_lang=None, task='translation', tensorboard_logdir='', testpref='/home/jupyter/data/fairseq/evaluation/en/en.txt', tgtdict=None, threshold_loss_scale=None, threshol

#### en vs zh

In [13]:
%env tgt_lang=zh
# Processes data (only --testpref)
! fairseq-preprocess --only-source --testpref $outpath/$tgt_lang.txt --destdir $destdir/$src_lang-$tgt_lang --workers 20 --joined-dictionary --srcdict $modelpath/dict.txt
# run evalution
! fairseq-eval-lm $destdir/$src_lang-$tgt_lang --path $modelpath/model.pt --max-sentences 2 --tokens-per-sample 512 --context-window 400

env: tgt_lang=zh
2020-05-31 15:16:14 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, bf16=False, bpe=None, checkpoint_suffix='', cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='/home/jupyter/fairseq_data/en/en-zh', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=True, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=True, optimizer='nag', padding_factor=8, quantization_config_path=None, seed=1, source_lang=None, srcdict='/home/jupyter/fairseq_models/wmt19.en/dict.txt', target_lang=None, task='translation', tensorboard_logdir='', testpref='/home/jupyter/data/fairseq/evaluation/en/zh.txt', tgtdict=None, threshold_loss_scale=None, threshol

#### en vs fr

In [14]:
%env tgt_lang=fr
# Processes data (only --testpref)
! fairseq-preprocess --only-source --testpref $outpath/$tgt_lang.txt --destdir $destdir/$src_lang-$tgt_lang --workers 20 --joined-dictionary --srcdict $modelpath/dict.txt
# run evalution
! fairseq-eval-lm $destdir/$src_lang-$tgt_lang --path $modelpath/model.pt --max-sentences 2 --tokens-per-sample 512 --context-window 400

env: tgt_lang=fr
2020-05-31 15:20:29 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, bf16=False, bpe=None, checkpoint_suffix='', cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='/home/jupyter/fairseq_data/en/en-fr', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=True, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=True, optimizer='nag', padding_factor=8, quantization_config_path=None, seed=1, source_lang=None, srcdict='/home/jupyter/fairseq_models/wmt19.en/dict.txt', target_lang=None, task='translation', tensorboard_logdir='', testpref='/home/jupyter/data/fairseq/evaluation/en/fr.txt', tgtdict=None, threshold_loss_scale=None, threshol

#### en vs de

In [15]:
%env tgt_lang=de
# Processes data (only --testpref)
! fairseq-preprocess --only-source --testpref $outpath/$tgt_lang.txt --destdir $destdir/$src_lang-$tgt_lang --workers 20 --joined-dictionary --srcdict $modelpath/dict.txt
# run evalution
! fairseq-eval-lm $destdir/$src_lang-$tgt_lang --path $modelpath/model.pt --max-sentences 2 --tokens-per-sample 512 --context-window 400

env: tgt_lang=de
2020-05-31 16:25:31 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, bf16=False, bpe=None, checkpoint_suffix='', cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='/home/jupyter/fairseq_data/en/en-de', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=True, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=True, optimizer='nag', padding_factor=8, quantization_config_path=None, seed=1, source_lang=None, srcdict='/home/jupyter/fairseq_models/wmt19.en/dict.txt', target_lang=None, task='translation', tensorboard_logdir='', testpref='/home/jupyter/data/fairseq/evaluation/en/de.txt', tgtdict=None, threshold_loss_scale=None, threshol

#### en vs it

In [16]:
%env tgt_lang=it
# Processes data (only --testpref)
! fairseq-preprocess --only-source --testpref $outpath/$tgt_lang.txt --destdir $destdir/$src_lang-$tgt_lang --workers 20 --joined-dictionary --srcdict $modelpath/dict.txt
# run evalution
! fairseq-eval-lm $destdir/$src_lang-$tgt_lang --path $modelpath/model.pt --max-sentences 2 --tokens-per-sample 512 --context-window 400

env: tgt_lang=it
2020-05-31 17:47:50 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, bf16=False, bpe=None, checkpoint_suffix='', cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='/home/jupyter/fairseq_data/en/en-it', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=True, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=True, optimizer='nag', padding_factor=8, quantization_config_path=None, seed=1, source_lang=None, srcdict='/home/jupyter/fairseq_models/wmt19.en/dict.txt', target_lang=None, task='translation', tensorboard_logdir='', testpref='/home/jupyter/data/fairseq/evaluation/en/it.txt', tgtdict=None, threshold_loss_scale=None, threshol

#### en vs ru

In [17]:
%env tgt_lang=ru
# Processes data (only --testpref)
! fairseq-preprocess --only-source --testpref $outpath/$tgt_lang.txt --destdir $destdir/$src_lang-$tgt_lang --workers 20 --joined-dictionary --srcdict $modelpath/dict.txt
# run evalution
! fairseq-eval-lm $destdir/$src_lang-$tgt_lang --path $modelpath/model.pt --max-sentences 2 --tokens-per-sample 512 --context-window 400

env: tgt_lang=ru
2020-05-31 18:37:41 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, bf16=False, bpe=None, checkpoint_suffix='', cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='/home/jupyter/fairseq_data/en/en-ru', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=True, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=True, optimizer='nag', padding_factor=8, quantization_config_path=None, seed=1, source_lang=None, srcdict='/home/jupyter/fairseq_models/wmt19.en/dict.txt', target_lang=None, task='translation', tensorboard_logdir='', testpref='/home/jupyter/data/fairseq/evaluation/en/ru.txt', tgtdict=None, threshold_loss_scale=None, threshol

#### en vs es

In [18]:
%env tgt_lang=es
# Processes data (only --testpref)
! fairseq-preprocess --only-source --testpref $outpath/$tgt_lang.txt --destdir $destdir/$src_lang-$tgt_lang --workers 20 --joined-dictionary --srcdict $modelpath/dict.txt
# run evalution
! fairseq-eval-lm $destdir/$src_lang-$tgt_lang --path $modelpath/model.pt --max-sentences 2 --tokens-per-sample 512 --context-window 400

env: tgt_lang=es
2020-05-31 19:03:49 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, bf16=False, bpe=None, checkpoint_suffix='', cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='/home/jupyter/fairseq_data/en/en-es', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=True, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=True, optimizer='nag', padding_factor=8, quantization_config_path=None, seed=1, source_lang=None, srcdict='/home/jupyter/fairseq_models/wmt19.en/dict.txt', target_lang=None, task='translation', tensorboard_logdir='', testpref='/home/jupyter/data/fairseq/evaluation/en/es.txt', tgtdict=None, threshold_loss_scale=None, threshol